### Sentiment analysis toy project on tweets about Babylon - Part 2
#### Analysing tweets

<ol>
<li>Try other models to see what we get (spacy, allennlp ?)</li>
<li>Get proper scores, on more tweets once fetchig is fixed</li>
<li>Get some detailed visualisation of the results</li>
    <ul>
        <li>percentage of positive/negative/neutral tweets</li>
        <li>average confidence score for each category</li>
        <li>average subjectivity score for each category</li>
        <li>...</li>
    </ul>
</ol>

In [11]:
import json
import sys
import os
from monkeylearn import MonkeyLearn

In [18]:
class SentimentAnalyser:
    """ Using MonkeyLearn sentiment analysis model to get tweets polarity """
    def __init__(self, tweets):
        self.tweets = tweets                                           # Loaded raw tweets
        self.keys = json.load(open("../data/twitter_keys.json", "r"))  # API/model keys
        self.monkey_model = MonkeyLearn(self.keys['monkey_api_key'])   # model
        
    def analyse_tweets(self, force_reload=False):
        """ Perform sentiment analysis on tweets
        :param force_reload: if True, will call the model again 
                                (reducing # of available queries from MonkeyLearn)
        :returns classification results
        """
        if force_reload:
            return self.monkey_model.classifiers.classify(self.keys['model_id'], self.tweets)
        else:
            if os.path.exists("../data/analysed_tweets.json"):
                try:
                    return json.load(open("../data/analysed_tweets.json", "r"))
                except ValueError:
                    return self.monkey_model.classifiers.classify(self.keys['model_id'], self.tweets)
            else:
                return self.monkey_model.classifiers.classify(self.keys['model_id'], self.tweets)
    
    def get_polarity_score(self, tweet):
        return TextBlob(tweet).polarity
    
    def get_sentiment_score(self, tweet):
        return TextBlob(tweet).sentiment
    
    def show_raw_results(self):
        """ Prints base results returned from MonkeyLearn/TextBlob """
        pos, neg, neut = 0, 0, 0  # positive, negative and neutral tweets, according to model
        for at in self.analysed_tweets.body:
            print(at['text'])
            print("Polarity", sa.get_polarity_score(at['text']))
            print("TB sentiment", sa.get_sentiment_score(at['text']))
            for elem in at['classifications']:
                if elem['tag_name'] == "Neutral":
                    neut += 1
                elif elem['tag_name'] == "Positive":
                    pos += 1
                else:
                    neg += 1
                print(elem['tag_name'], elem['confidence'])
            print("-" * 20)
        
        print("{} positive tweets".format(pos))
        print("{} negative tweets".format(neg))
        print("{} neutral tweets".format(neut))
    
    # TODO
    def visualise(self):
        pass

In [ ]:
try:
    tweets = json.load(open("../data/raw_tweets.json", "r"))
except IOError:
    sys.exit("'raw_tweets.json' doesn't exist !")
    
sa = SentimentAnalyser(tweets)
analysed = sa.analyse_tweets()
sa.show_raw_results()